Code lấy tên + link phim

In [6]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os

# Hàm đọc danh sách phim từ CSV vào set
def load_existing_movies(csv_filename):
    movie_set = set()
    if os.path.exists(csv_filename):
        with open(csv_filename, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader, None)  # Bỏ qua header nếu có
            for row in reader:
                movie_set.add(row[0])
    return movie_set

# Cấu hình Selenium
driver = webdriver.Chrome()
driver.get("https://www.momo.vn/cinema/review")  
time.sleep(3)  # Đợi trang load

# Bấm nút "Xem thêm" nếu có và kiểm tra trong CSV mỗi 2 lần bấm
wait = WebDriverWait(driver, 5)
click_count = 0
csv_filename = "movies.csv"
existing_movies = load_existing_movies(csv_filename)
movies_to_save = []

for i in range(1000):  # Giới hạn số lần bấm
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Xem tiếp nhé !')]")))
        button.click()
        click_count += 1
        time.sleep(2)  # Đợi nội dung mới tải về

        # Mỗi 2 lần bấm thì kiểm tra và lấy dữ liệu
        if click_count % 2 == 0:
            page_html = driver.page_source
            soup = BeautifulSoup(page_html, 'html.parser')
            movie_divs = soup.find_all('div', class_='relative h-full overflow-hidden rounded-xl border border-gray-200 bg-white shadow-lg')
            new_count = 0
            
            for movie in movie_divs:
                title_tag = movie.find('div', class_='truncate text-sm font-bold leading-tight text-white hover:text-pink-100')
                movie_name = title_tag.text.strip() if title_tag else "Không rõ"
                review_link_tag = movie.find('a', href=True)
                movie_link = "https://www.momo.vn" + review_link_tag['href'] if review_link_tag else "Không có link"
                
                if movie_name not in existing_movies:
                    existing_movies.add(movie_name)
                    movies_to_save.append((movie_name, movie_link))
                    new_count += 1
            
            if new_count > 0:
                print(f"Lưu {new_count} phim mới sau {click_count} lần bấm")
    except:
        break  # Nếu không tìm thấy nút, thoát vòng lặp

# Đóng trình duyệt
driver.quit()

# Ghi danh sách phim vào file CSV nếu có phim mới
if movies_to_save:
    file_exists = os.path.exists(csv_filename)
    with open(csv_filename, mode='a' if file_exists else 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Tên phim", "Link review"])  # Ghi header nếu file chưa tồn tại
        writer.writerows(movies_to_save)
    print(f"Tổng số phim mới được lưu: {len(movies_to_save)}")
else:
    print("Không có phim mới để lưu.")

print(f"Số lần bấm nút: {click_count}")

Lưu 6 phim mới sau 12 lần bấm
Lưu 6 phim mới sau 14 lần bấm
Lưu 6 phim mới sau 16 lần bấm
Lưu 6 phim mới sau 18 lần bấm
Lưu 6 phim mới sau 20 lần bấm
Lưu 6 phim mới sau 22 lần bấm
Lưu 6 phim mới sau 24 lần bấm
Lưu 6 phim mới sau 26 lần bấm
Lưu 6 phim mới sau 28 lần bấm
Lưu 6 phim mới sau 30 lần bấm
Lưu 6 phim mới sau 32 lần bấm
Lưu 6 phim mới sau 34 lần bấm
Lưu 6 phim mới sau 36 lần bấm
Lưu 6 phim mới sau 38 lần bấm
Lưu 6 phim mới sau 40 lần bấm
Lưu 6 phim mới sau 42 lần bấm
Lưu 6 phim mới sau 44 lần bấm
Lưu 6 phim mới sau 46 lần bấm
Lưu 6 phim mới sau 48 lần bấm
Lưu 6 phim mới sau 50 lần bấm
Lưu 6 phim mới sau 52 lần bấm
Lưu 6 phim mới sau 54 lần bấm
Lưu 6 phim mới sau 56 lần bấm
Lưu 6 phim mới sau 58 lần bấm
Lưu 6 phim mới sau 60 lần bấm
Lưu 6 phim mới sau 62 lần bấm
Lưu 6 phim mới sau 64 lần bấm
Lưu 6 phim mới sau 66 lần bấm
Lưu 6 phim mới sau 68 lần bấm
Lưu 6 phim mới sau 70 lần bấm
Lưu 6 phim mới sau 72 lần bấm
Lưu 6 phim mới sau 74 lần bấm
Lưu 6 phim mới sau 76 lần bấm
Lưu 6 phim

Code lấy thể loại phim + link hình

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# Đọc file CSV
csv_filename = "movies.csv"
df = pd.read_csv(csv_filename)

# Thêm cột "Thể loại" và "Ảnh" nếu chưa có
if "Thể loại" not in df.columns:
    df["Thể loại"] = ""
if "Ảnh" not in df.columns:
    df["Ảnh"] = ""

# Mở trình duyệt Selenium
driver = webdriver.Chrome()

# Duyệt qua từng dòng trong CSV
for index, row in df.iterrows():
    link = row[1]  # Cột 1 chứa link review

    if pd.isna(link) or link == "":  # Bỏ qua nếu link trống
        continue

    try:
        driver.get(link)
        time.sleep(2)  # Đợi trang load

        # Dùng BeautifulSoup để lấy dữ liệu từ trang
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Lấy thể loại
        li_tags = soup.find_all('li', class_="jsx-d074b6b0f0aeffcc")
        first_genre_li = [li for li in li_tags if "Thể loại" in li.get_text()]
        if first_genre_li:
            genre = first_genre_li[0].get_text(strip=True).split(":")[-1].strip()
            df.loc[index, "Thể loại"] = genre
            print(f"{index + 1}/{len(df)} - Cập nhật thể loại: {genre}")

        # Lấy ảnh
        div_tag = soup.find('div', class_="jsx-d074b6b0f0aeffcc relative z-[1] mb-3 aspect-[21/9] w-full overflow-hidden rounded-md bg-gray-200 bg-cover bg-no-repeat md:mb-5 md:aspect-[3/1] md:rounded-xl")
        if div_tag and "style" in div_tag.attrs:
            style = div_tag["style"]
            match = re.search(r'url\((.*?)\)', style)  # Tách link ảnh
            if match:
                image_url = match.group(1)
                df.loc[index, "Ảnh"] = image_url
                print(f"{index + 1}/{len(df)} - Cập nhật ảnh: {image_url}")

    except Exception as e:
        print(f"Lỗi ở dòng {index + 1}: {e}")

# Đóng trình duyệt sau khi xong
driver.quit()

# Ghi lại file CSV
df.to_csv(csv_filename, index=False)

print("Hoàn thành cập nhật toàn bộ dữ liệu!")


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7724\373337600.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link = row[1]  # Cột 1 chứa link review


1/845 - Cập nhật thể loại: Gia đình, Tâm Linh, Hài
1/845 - Cập nhật ảnh: https://cinema.momocdn.net/img/30910758424122498-672b4df6707cf_1.jpg?size=L


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7724\373337600.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link = row[1]  # Cột 1 chứa link review


2/845 - Cập nhật thể loại: Hài, Lãng mạn
2/845 - Cập nhật ảnh: https://cinema.momocdn.net/img/20805460312366259-dsc07289-5246.jpg?size=L


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7724\373337600.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link = row[1]  # Cột 1 chứa link review


...

843/845 - Cập nhật thể loại: Gay cấn, Kinh dị
843/845 - Cập nhật ảnh: https://homepage.momocdn.net/cinema/momo-cdn-api-220615142423-637908998631066836.jpg


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7724\373337600.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link = row[1]  # Cột 1 chứa link review


844/845 - Cập nhật thể loại: Hoạt hình
844/845 - Cập nhật ảnh: https://homepage.momocdn.net/cinema/momo-cdn-api-220615142622-637908999829900844.jpg


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7724\373337600.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link = row[1]  # Cột 1 chứa link review


845/845 - Cập nhật thể loại: Âm Nhạc, Tâm lý, Tình cảm, Lãng mạn
845/845 - Cập nhật ảnh: https://homepage.momocdn.net/cinema/momo-cdn-api-220615142349-637908998290307009.jpg
Hoàn thành cập nhật toàn bộ dữ liệu!


Code lấy comment

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import unidecode
import os

# Đọc file CSV gốc
csv_filename = "movies.csv"
df = pd.read_csv(csv_filename)

# Khởi tạo Selenium WebDriver
driver = webdriver.Chrome()

# Số lần bấm "Xem tiếp nhé!"
time_press_see_more = 900

# Duyệt qua từng dòng trong CSV
for index, row in df.iterrows():
    ten_phim = row[0]  # Giả sử cột 0 là tên phim
    link = row[1]  # Cột 1 chứa link review

    # Chuyển tên phim thành file CSV không dấu, không cách
    file_name = unidecode.unidecode(ten_phim).replace(" ", "_") + ".csv"
    print(f"Đang lấy bình luận cho phim {ten_phim}...")

    # Đọc bình luận cũ nếu file tồn tại
    if os.path.exists(file_name):
        df_old = pd.read_csv(file_name)
        print(f"Đã tìm thấy file {file_name} chứa {len(df_old)} bình luận cũ.")
        old_comments = set(df_old["Bình luận"].astype(str))
    else:
        old_comments = set()

    try:
        driver.get(link)
        time.sleep(5)  # Đợi trang load
        print(f"Đã mở trang {link}")

        # Bấm nút "Xem tiếp nhé!" nhiều lần để tải hết bình luận
        for _ in range(time_press_see_more):
            try:
                xem_them_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem tiếp nhé!')]"))
                )
                ActionChains(driver).move_to_element(xem_them_button).click().perform()
                time.sleep(2)
            except:
                break  

        print("Đã bấm hết nút 'Xem tiếp nhé!'")

        # Bấm vào tất cả các nút "Xem thêm" nếu bình luận chứa "..."
        while True:
            try:
                xem_them_cmt_buttons = driver.find_elements(By.XPATH, "//span[contains(text(),'Xem thêm')]")
                if not xem_them_cmt_buttons:
                    break  # Không còn nút nào để bấm

                for button in xem_them_cmt_buttons:
                    parent_div = button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'whitespace-pre-wrap')]")
                    if "...Xem thêm" in parent_div.text:  # Chỉ bấm nếu có "..."
                        driver.execute_script("arguments[0].click();", button)
                        time.sleep(0.5)  # Đợi bình luận mở ra

                time.sleep(1)  # Chờ toàn bộ nội dung cập nhật
            except:
                break  

        print("Đã mở hết các bình luận bị ẩn.")

        # Dùng BeautifulSoup để lấy dữ liệu từ trang
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        print("Đã lấy dữ liệu từ trang")

        # Lấy tất cả các div chứa bình luận
        divs = soup.find_all('div', class_="text-md whitespace-pre-wrap break-words leading-relaxed text-gray-900")

        # Lọc ra các bình luận mới (tránh trùng lặp)
        new_comments = [div.get_text(strip=True) for div in divs if div.get_text(strip=True) not in old_comments]

        # Nếu có bình luận mới thì cập nhật file CSV
        if new_comments:
            df_new = pd.DataFrame({"Bình luận": new_comments})
            df_new.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
            print(f"Đã cập nhật {len(new_comments)} bình luận mới vào {file_name}.")
        else:
            print(f"Không có bình luận mới cho phim {ten_phim}.")

    except Exception as e:
        print(f"Lỗi ở phim {ten_phim}: {e}")

# Đóng trình duyệt sau khi xong
driver.quit()

print("Hoàn thành lấy bình luận và cập nhật file CSV!")


In [ ]:
https://github.com/WebMining2025/demo-repository.git